<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Notebooks/gat_modelling_with_breast_cancer_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Data

In [1]:
!wget https://zenodo.org/record/8232723/files/X_train.csv?download=1 -O X_train.csv

--2023-08-26 16:28:05--  https://zenodo.org/record/8232723/files/X_train.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44836667 (43M) [text/plain]
Saving to: ‘X_train.csv’

X_train.csv         100%[===================>]  42.76M   538KB/s    in 86s     

2023-08-26 16:29:33 (506 KB/s) - ‘X_train.csv’ saved [44836667/44836667]



In [2]:
!wget https://zenodo.org/record/8232723/files/X_test.csv?download=1 -O X_test.csv

--2023-08-26 16:29:33--  https://zenodo.org/record/8232723/files/X_test.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15058192 (14M) [text/plain]
Saving to: ‘X_test.csv’

X_test.csv          100%[===================>]  14.36M  2.92MB/s    in 5.5s    

2023-08-26 16:29:41 (2.60 MB/s) - ‘X_test.csv’ saved [15058192/15058192]



In [3]:
!wget https://zenodo.org/record/8232723/files/y_test.csv?download=1 -O y_test.csv

--2023-08-26 16:29:41--  https://zenodo.org/record/8232723/files/y_test.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5425 (5.3K) [text/plain]
Saving to: ‘y_test.csv’

y_test.csv          100%[===================>]   5.30K  --.-KB/s    in 0s      

2023-08-26 16:29:42 (2.66 GB/s) - ‘y_test.csv’ saved [5425/5425]



In [4]:
!wget https://zenodo.org/record/8232723/files/y_train.csv?download=1 -O y_train.csv

--2023-08-26 16:29:42--  https://zenodo.org/record/8232723/files/y_train.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16225 (16K) [text/plain]
Saving to: ‘y_train.csv’

y_train.csv         100%[===================>]  15.84K  --.-KB/s    in 0s      

2023-08-26 16:29:44 (225 MB/s) - ‘y_train.csv’ saved [16225/16225]



In [5]:
!wget https://zenodo.org/record/8232723/files/X_val.csv?download=1 -O X_val.csv

--2023-08-26 16:29:44--  https://zenodo.org/record/8232723/files/X_val.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14951235 (14M) [text/plain]
Saving to: ‘X_val.csv’

X_val.csv           100%[===================>]  14.26M   542KB/s    in 28s     

2023-08-26 16:30:13 (526 KB/s) - ‘X_val.csv’ saved [14951235/14951235]



In [6]:
!wget https://zenodo.org/record/8232723/files/y_val.csv?download=1 -O y_val.csv

--2023-08-26 16:30:14--  https://zenodo.org/record/8232723/files/y_val.csv?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5400 (5.3K) [text/plain]
Saving to: ‘y_val.csv’

y_val.csv           100%[===================>]   5.27K  --.-KB/s    in 0s      

2023-08-26 16:30:15 (2.82 GB/s) - ‘y_val.csv’ saved [5400/5400]



In [7]:
!wget https://zenodo.org/record/8233085/files/edge_index.pt?download=1 -O edge_index.pt

--2023-08-26 16:30:15--  https://zenodo.org/record/8233085/files/edge_index.pt?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4349044 (4.1M) [application/octet-stream]
Saving to: ‘edge_index.pt’

edge_index.pt       100%[===================>]   4.15M   535KB/s    in 8.1s    

2023-08-26 16:30:24 (523 KB/s) - ‘edge_index.pt’ saved [4349044/4349044]



In [8]:
!pip install torch-geometric

# Importing Data and Libraries

In [9]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type != 'cuda':
    raise SystemError('GPU device not found')

Using device: cuda


In [11]:
X_train = pd.read_csv("X_train.csv", index_col=0)
y_train = np.loadtxt("y_train.csv", delimiter=',')

X_test = pd.read_csv("X_test.csv", index_col=0)
y_test = np.loadtxt("y_test.csv", delimiter=',')

X_val = pd.read_csv("X_val.csv", index_col=0)
y_val = np.loadtxt("y_val.csv", delimiter=',')

In [12]:
edge_index = torch.load('edge_index.pt')

In [13]:
edge_index

tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])

In [14]:
edge_index.shape

torch.Size([2, 271771])

In [15]:
X_train.shape, X_test.shape, X_val.shape

((649, 9288), (217, 9288), (216, 9288))

In [16]:
X_train = X_train.values
X_test = X_test.values
X_val = X_val.values

# Generating patient-specific graphs

In [17]:
num_patients_train = X_train.shape[0]
num_patients_test = X_test.shape[0]
num_patients_val = X_val.shape[0]

# Create patient-specific graphs for the training set
graphs_train = []
for i in range(num_patients_train):
    node_features = X_train[i]  # Node features for the i-th patient
    target = y_train[i]  # Target label for the i-th patient
    graph_train = (node_features, edge_index, target)
    graphs_train.append(graph_train)

# Create patient-specific graphs for the test set
graphs_test = []
for i in range(num_patients_test):
    node_features = X_test[i]  # Node features for the i-th patient
    target = y_test[i]  # Target label for the i-th patient
    graph_test = (node_features, edge_index, target)
    graphs_test.append(graph_test)

# Create patient-specific graphs for the test set
graphs_val = []
for i in range(num_patients_val):
    node_features = X_val[i]  # Node features for the i-th patient
    target = y_val[i]  # Target label for the i-th patient
    graph_val = (node_features, edge_index, target)
    graphs_val.append(graph_test)

In [18]:
# Check the number of patient-specific graphs
print(len(graphs_train))  # Should be 857
print(len(graphs_test))  # Should be 217
print(len(graphs_val))

649
217
216


In [19]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 0 # Index of the patient
node_features, edge_index, y = graphs_train[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
print(y)

[   0.     5798.37      8.6165 ...  415.824   931.957  1180.46  ]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
133.0505967


In [20]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 15  # Index of the patient
node_features, edge_index, y = graphs_test[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
# print(y)

[0.00000e+00 2.87393e+04 9.46600e-01 ... 5.60854e+02 7.04274e+02
 5.87369e+02]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])


In [21]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 15  # Index of the patient
node_features, edge_index, y = graphs_val[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
# print(y)

[   0.    7339.17     0.    ...  521.169 2435.84   462.114]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])


# Converting List of Graphs to Data Objects

In [22]:
# Convert graphs_train to a list of Data objects
data_train = [Data(x=torch.tensor(graph[0].reshape(len(graphs_train[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_train]

# Convert graphs_test to a list of Data objects
data_test = [Data(x=torch.tensor(graph[0].reshape(len(graphs_test[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_test]

# Convert graphs_test to a list of Data objects
data_val = [Data(x=torch.tensor(graph[0].reshape(len(graphs_val[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_val]

In [23]:
# Access the attributes of a specific data object in the training set
sample = data_train[0]  # Get the first data object
print(sample)  # Print the data object

# Access the node features, edge indices, and target label
node_features = sample.x
edge_index = sample.edge_index
target = sample.y

print(node_features)  # Print the node features
print(edge_index)  # Print the edge indices
print(target)  # Print the target label

Data(x=[9288, 1], edge_index=[2, 271771], y=133.0505967)
tensor([[   0.0000],
        [5798.3700],
        [   8.6165],
        ...,
        [ 415.8240],
        [ 931.9570],
        [1180.4600]], dtype=torch.float64)
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
tensor(133.0506, dtype=torch.float64)


# Creating Train, Test and Validation Batches

In [24]:
from torch_geometric.loader import DataLoader

In [25]:
bs = 8
train_loader = DataLoader(data_train, batch_size=bs, shuffle=True)
test_loader = DataLoader(data_test, batch_size=bs, shuffle=False)
val_loader = DataLoader(data_val, batch_size=bs, shuffle=True)

for step, data in enumerate(train_loader):
    data = data.to(device)  # Move the batch of data to the device

    print('Training Batches: ')
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Training Batches: 
Step 1:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Training Batches: 
Step 2:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Training Batches: 
Step 3:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Training Batches: 
Step 4:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Training Batches: 
Step 5:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Training Batches: 
Step 6:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Training Batches: 
Step 7:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2,

In [26]:
for step, data in enumerate(test_loader):
    data = data.to(device)
    print('Test Batches: ')
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Test Batches: 
Step 1:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Test Batches: 
Step 2:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Test Batches: 
Step 3:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Test Batches: 
Step 4:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Test Batches: 
Step 5:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Test Batches: 
Step 6:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Test Batches: 
Step 7:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[743

In [27]:
for step, data in enumerate(val_loader):
    data = data.to(device)
    print('Val Batches: ')
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Val Batches: 
Step 1:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Val Batches: 
Step 2:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Val Batches: 
Step 3:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Val Batches: 
Step 4:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Val Batches: 
Step 5:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Val Batches: 
Step 6:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Val Batches: 
Step 7:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], pt

# Model Building and Evaluation

In [28]:
import torch_geometric
from torch.nn import Linear, Module, Dropout, MSELoss, CrossEntropyLoss, BatchNorm1d
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GraphNorm
from torch_geometric.nn import global_mean_pool

from torch.nn.functional import relu, sigmoid
# from torch_sparse import SparseTensor

In [29]:
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_heads, dropout):
        super(GAT, self).__init__()
        self.gat1 = GATConv(num_node_features, hidden_channels, heads=num_heads)
        self.gat2 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads)
        self.gat3 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads)

        self.dropout = Dropout(dropout)

        self.graph_norm1 = GraphNorm(hidden_channels * num_heads)
        self.graph_norm2 = GraphNorm(hidden_channels * num_heads)
        self.graph_norm3 = GraphNorm(hidden_channels * num_heads)

        self.linear1 = Linear(hidden_channels * num_heads + hidden_channels * num_heads, 1)

    def forward(self, x, edge_index, batch):
          x = self.gat1(x, edge_index)
          x = self.dropout(relu(self.graph_norm1(x)))
          x = self.gat2(x, edge_index, return_attention_weights=False)
          x = self.dropout(relu(self.graph_norm1(x)))
          x = self.gat3(x, edge_index, return_attention_weights=False)
          x = self.dropout(relu(self.graph_norm1(x)))

          return self.linear1(x).squeeze()


In [30]:
class GAT(torch.nn.Module):
    def __init__(self, num_node_features, in_heads, hidden_channels, out_head, dropout):
        super(GAT, self).__init__()


        self.conv1 = GATConv(num_node_features, hidden_channels, heads=in_heads)
        self.conv2 = GATConv(hidden_channels * in_heads, hidden_channels, heads=in_heads)
        self.conv3 = GATConv(hidden_channels * in_heads, 1, concat=False,
                             heads=out_head)

    def forward(self, x, edge_index, batch):
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv3(x, edge_index)

        return x.squeeze()

In [31]:
import time

In [32]:
# Assuming you have train_loader, val_loader, and test_loader DataLoader instances
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT(num_node_features=1, in_heads=2, hidden_channels=64, out_head=1, dropout=0.2)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

model = model.to(device)  # Move the model to CUDA device
criterion = criterion.to(device)  # Move the criterion to CUDA device

model.double()

num_epochs = 101

start_time = time.time()

# Initialize lists to store training and test losses
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, data in enumerate(train_loader):
      data = data.to(device)
      optimizer.zero_grad()

      out = model(data.x.double(), data.edge_index, data.batch)
      loss = criterion(out, data.y.view(-1, 1).to(device))
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    train_losses.append(average_loss)

    model.eval()
    with torch.no_grad():
        val_loss = 0
        for data in val_loader:
            data = data.to(device)

            out = model(data.x.double(), data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1, 1).to(device))
            val_loss += loss.item()
            val_losses.append(val_loss)


        average_val_loss = val_loss / len(val_loader)
        print(f'Epoch: {epoch:03d}, Train loss: {average_loss:.4f}, Validation Loss: {average_val_loss:.4f}')

# Calculate the elapsed time
elapsed_time = time.time() - start_time

print(f"Time used for training: {elapsed_time:.2f} seconds")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([74304])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([9288])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 000, Train loss: 5068305.1407, Validation Loss: 11353.7261
Epoch: 001, Train loss: 27201.0449, Validation Loss: 10093.2762
Epoch: 002, Train loss: 23409.4879, Validation Loss: 10274.1549
Epoch: 003, Train loss: 20792.3231, Validation Loss: 10329.1512
Epoch: 004, Train loss: 11319.5594, Validation Loss: 12147.0201
Epoch: 005, Train loss: 11819.8225, Validation Loss: 11933.0276
Epoch: 006, Train loss: 8288.0799, Validation Loss: 11328.4694
Epoch: 007, Train loss: 8552.9466, Validation Loss: 11364.9544
Epoch: 008, Train loss: 9184.9579, Validation Loss: 11255.9925
Epoch: 009, Train loss: 8933.4675, Validation Loss: 11261.2361
Epoch: 010, Train loss: 6528.0330, Validation Loss: 10815.1082
Epoch: 011, Train loss: 6433.6275, Validation Loss: 10523.1689
Epoch: 012, Train loss: 5524.9716, Validation Loss: 9892.5426
Epoch: 013, Train loss: 5078.9345, Validation Loss: 9246.1848
Epoch: 014, Train loss: 4768.1572, Validation Loss: 8474.1607
Epoch: 015, Train loss: 4391.6666, Validation Loss

## Making Predictions and calculating MSE

In [33]:
model.eval()  # Set the model to evaluation mode

predictions = []  # List to store the predicted outputs
test_losses = []
with torch.no_grad():
    test_loss = 0
    for data in test_loader:
        data = data.to(device)  # Move the batch of data to CUDA device
        out = model(data.x.double(), data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1).double())
        test_loss += loss.item()

        average_test_loss = test_loss / len(test_loader)
        test_losses.append(average_val_loss)

        predictions.append(out.cpu().detach().numpy())  # Convert the predictions to NumPy array

# Concatenate the predictions from multiple batches
predictions = np.concatenate(predictions)

# Print the predictions
print(predictions[:10])

[41.29224279  5.98579127 41.67366348 41.49045081 41.44243904 41.17263439
 41.45575732 41.13383185 41.7304018  41.34804862]


In [34]:
# Convert the test data batches to a list of Data objects
test_data_batches = [
    Data(x=batch.x, edge_index=batch.edge_index, y=batch.y) for batch in test_loader
]

# Convert the predictions to PyTorch tensors
predictions = torch.tensor(predictions)

# Convert the ground truth labels of the test data to a PyTorch tensor
y_true = torch.cat([batch.y for batch in test_data_batches])

# Calculate the mean squared error (MSE) loss using PyTorch's function
mse_loss = torch.nn.functional.mse_loss(predictions.view(-1), y_true.view(-1, 1))

print(f"Mean Squared Error (MSE) Loss: {mse_loss:.4f}")

<ipython-input-34-7db24328f723>:13: UserWarning: Using a target size (torch.Size([217, 1])) that is different to the input size (torch.Size([2015496])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse_loss = torch.nn.functional.mse_loss(predictions.view(-1), y_true.view(-1, 1))


Mean Squared Error (MSE) Loss: 916.8123
